# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00


In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [ ]:
diamonds.shape

(53940, 10)

In [5]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [6]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [ ]:
dtrain

In [ ]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:49:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:54:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [ ]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [ ]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:54:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[20]	train-rmse:491.51435	validation-rmse:544.08058
[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [ ]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:01:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[102]	train-rmse:376.20429	validation-rmse:532.59813


In [ ]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:03:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [ ]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [ ]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [7]:
# Log in to your W&B account
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
params = [{
    'max_depth': 3,
    'learning_rate': 0.01,
    'n_estimators': 500,
    'objective': 'reg:squarederror',
}, {
    'max_depth': 4,
    'learning_rate': 0.01,
    'n_estimators': 1500,
    'objective': 'reg:squarederror',
}, {
    'max_depth': 5,
    'learning_rate': 0.01,
    'n_estimators': 2000,
    'objective': 'reg:squarederror',
}, {
    'max_depth': 6,
    'learning_rate': 0.01,
    'n_estimators': 2500,
    'objective': 'reg:squarederror',
}, {
    'max_depth': 7,
    'learning_rate': 0.01,
    'n_estimators': 3000,
    'objective': 'reg:squarederror',
}]

In [12]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see
import random
from wandb.xgboost import WandbCallback

runs = 5

for run in range(runs):
  wandb.init(
      project="homework",
      name = f'experiment_{run}'
      )

  bst = xgb.train(params[run],
                dtrain=xgb.DMatrix(X_train, label=y_train, enable_categorical=True),
                num_boost_round=1000,
                early_stopping_rounds=10,
                evals=[(xgb.DMatrix(X_test, label=y_test, enable_categorical=True), 'test')],
                callbacks=[WandbCallback(log_model=True)])

  wandb.log({"mse": bst.best_score})

  wandb.finish()

[0]	test-rmse:3951.56348
[1]	test-rmse:3916.31590
[2]	test-rmse:3881.43992
[3]	test-rmse:3846.96182
[4]	test-rmse:3812.85403
[5]	test-rmse:3779.13717
[6]	test-rmse:3745.77761
[7]	test-rmse:3712.80584
[8]	test-rmse:3680.18539
[9]	test-rmse:3647.95425
[10]	test-rmse:3616.06385
[11]	test-rmse:3584.53089
[12]	test-rmse:3553.34097
[13]	test-rmse:3522.51072
[14]	test-rmse:3491.99869
[15]	test-rmse:3461.89004
[16]	test-rmse:3432.08215
[17]	test-rmse:3402.60651
[18]	test-rmse:3373.53652
[19]	test-rmse:3344.72442
[20]	test-rmse:3316.26917
[21]	test-rmse:3288.14795


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:16:49] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[22]	test-rmse:3260.21661
[23]	test-rmse:3232.67351
[24]	test-rmse:3205.49100
[25]	test-rmse:3178.61325
[26]	test-rmse:3151.89647
[27]	test-rmse:3125.30611
[28]	test-rmse:3099.25897
[29]	test-rmse:3073.52870
[30]	test-rmse:3047.80070
[31]	test-rmse:3022.54974
[32]	test-rmse:2997.69030
[33]	test-rmse:2972.96232
[34]	test-rmse:2948.25363
[35]	test-rmse:2924.20676
[36]	test-rmse:2900.43908
[37]	test-rmse:2876.54729
[38]	test-rmse:2853.27017
[39]	test-rmse:2830.31620
[40]	test-rmse:2807.30479
[41]	test-rmse:2784.70041
[42]	test-rmse:2762.00483
[43]	test-rmse:2740.04878
[44]	test-rmse:2718.36920
[45]	test-rmse:2696.64481
[46]	test-rmse:2675.39073
[47]	test-rmse:2653.88461
[48]	test-rmse:2633.12208
[49]	test-rmse:2612.56634
[50]	test-rmse:2591.85875
[51]	test-rmse:2571.86932
[52]	test-rmse:2551.98421
[53]	test-rmse:2531.97535
[54]	test-rmse:2512.58999
[55]	test-rmse:2493.46945
[56]	test-rmse:2474.37276
[57]	test-rmse:2455.63809
[58]	test-rmse:2436.68943
[59]	test-rmse:2418.36295
[60]	test-rm

best_iteration,▁
best_score,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
mse,▁
test-rmse,█▆▅▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,999
best_score,612.7514
epoch,999
mse,612.7514


[0]	test-rmse:3950.61773
[1]	test-rmse:3914.40890
[2]	test-rmse:3878.59368
[3]	test-rmse:3843.17017
[4]	test-rmse:3808.12555
[5]	test-rmse:3773.40305
[6]	test-rmse:3739.11524
[7]	test-rmse:3705.13760
[8]	test-rmse:3671.58544
[9]	test-rmse:3638.33282
[10]	test-rmse:3605.26862
[11]	test-rmse:3572.61777
[12]	test-rmse:3540.35328
[13]	test-rmse:3508.40108
[14]	test-rmse:3476.83374
[15]	test-rmse:3445.69141
[16]	test-rmse:3414.80377
[17]	test-rmse:3384.22173
[18]	test-rmse:3353.78426


...

best_iteration,▁
best_score,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
mse,▁
test-rmse,█▇▅▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,899
best_score,525.84698
epoch,908
mse,525.84698


[0]	test-rmse:3948.83893
[1]	test-rmse:3910.84403
[2]	test-rmse:3873.22824
[3]	test-rmse:3836.01979
[4]	test-rmse:3799.18495
[5]	test-rmse:3762.62853


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:18:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[6]	test-rmse:3726.54841
[7]	test-rmse:3690.72264
[8]	test-rmse:3655.39667
[9]	test-rmse:3620.29136
[10]	test-rmse:3585.61493
[11]	test-rmse:3551.25891
[12]	test-rmse:3517.31286
[13]	test-rmse:3483.62884
[14]	test-rmse:3450.37804
[15]	test-rmse:3417.50993
[16]	test-rmse:3384.91214
[17]	test-rmse:3352.81323
[18]	test-rmse:3320.76934
[19]	test-rmse:3289.16547
[20]	test-rmse:3257.93436
[21]	test-rmse:3227.11559
[22]	test-rmse:3196.58000
[23]	test-rmse:3166.21053
[24]	test-rmse:3136.33933
[25]	test-rmse:3106.77581
[26]	test-rmse:3077.49524
[27]	test-rmse:3048.39741
[28]	test-rmse:3019.69354
[29]	test-rmse:2991.33067
[30]	test-rmse:2963.27229
[31]	test-rmse:2935.48572
[32]	test-rmse:2907.84713
[33]	test-rmse:2880.69801
[34]	test-rmse:2853.78395
[35]	test-rmse:2827.08787
[36]	test-rmse:2800.66710
[37]	test-rmse:2774.53590
[38]	test-rmse:2748.52153
[39]	test-rmse:2722.91346
[40]	test-rmse:2697.61936
[41]	test-rmse:2672.56704
[42]	test-rmse:2647.83742
[43]	test-rmse:2623.14039
[44]	test-rmse:2

best_iteration,▁
best_score,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
mse,▁
test-rmse,█▇▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,911
best_score,527.08386
epoch,921
mse,527.08386


The results show the performance of the XGBoost model trained with different hyperparameter settings. Each parameter configuration, characterized by variations in 'max_depth' and 'n_estimators', underwent training and evaluation, resulting in the reported best iteration and corresponding best score. The best iteration represents the round at which early stopping occurred, indicating the optimal stopping point during training. The best score, on the other hand, reflects the model's predictive performance, likely measured using a validation set. Notably, as the depth of the trees ('max_depth') increased, the best score generally decreased, suggesting that deeper trees yielded better predictive performance. Similarly, increasing the number of estimators ('n_estimators') led to improved performance, albeit with diminishing returns beyond a certain point. These findings underscore the importance of balancing model complexity and computational resources to achieve optimal performance in XGBoost models. Further experimentation and fine-tuning of hyperparameters could potentially enhance the model's predictive capabilities for the given task.